# Clustering & Summarization

### 필요 라이브러리 임포트

In [59]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from collections import Counter
from transformers import *
from summarizer import Summarizer
from sklearn.cluster import KMeans

### 데이터 불러오기

In [48]:
# 최종 기사 선별 후, Call 하기 위해 두가지 버전 의 dataframe 생성

#------ Original------
dforigin = pd.read_csv('./data/final_df.csv')

#------ High_ Simcount------
dffine = df[df['simcount'] > 4] # 유사한 기사의 수가 4개이상 있는 기사들만 출력 
dffine

,media,date,title,article_original,url,tokenized_sents,sim,simcount
14,조선일보,2020-01-15,현대미술과 손 잡은 BTS,14일현지 시각 영국 런던 유명 미술관 서펜타인갤러리에 구름 인파가 모였다. 다양...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['영국', '런던', '유명', '미술관', '펜', '갤러리', '구름', '인...","38, 36, 37, 23, 16",5
16,한겨레,2020-01-15,금속선으로 연결된 미술 거장 앤터니 곰리와 방탄소년단,거장으로 추앙받는 영국의 노장 조각가 앤터니 곰리70가 지난 14일 런던 서펀타인...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['거장', '추앙', '영국', '노장', '조각가', '앤터니', '곰리', '...","23, 14, 36, 38, 37",5
19,조선일보,2020-01-17,방탄소년단 신보 수록곡 선공개 전 세계 아미 시선 집중,방탄소년단BTS이 정규 앨범 발매에 앞서 수록곡 중 하나를 먼저 선보인다. 지난해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['방탄소년단', '정규', '앨범', '발매', '앞서', '수록', '곡', '...","20, 15, 35, 8, 7",5
20,조선일보,2020-01-17,방탄소년단 새앨범 선공개곡 블랙 스완 깊은 내면의 자전적 가사,방탄소년단은 이날 오후 6시 국내 및 해외 주요 음원사이트를 통해 정규 4집 ‘맵...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['방탄소년단', '날', '오후', '국내', '해외', '주요', '음원', '...","41, 40, 21, 19, 35, 42",6
21,한겨레,2020-01-20,방탄소년단 블랙스완 93개국 아이튠즈 1위K팝 최다,방탄 소속사 빅히트엔터테인먼트는 ‘블랙스완’이 18일오전 9시까지 기준으로 세계 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['방탄', '소속사', '빅히트', '엔터테인먼트', '블랙스완', '기준', '...","20, 42, 41, 40, 39",5
...,...,...,...,...,...,...,...,...
726,경향신문,2020-12-11,그들은 세상에서 가장 위대한 밴드가 되었다BTS 타임 올해의 연예인으로 선정,타임은 10일현지시간 “BTS는 이제 ‘음악 차트 상에서의 가장 큰 성취’에 그치...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['타임', '이제', '음악', '차트', '상', '가장', '성취', '세상'...","724, 728, 730, 722, 697, 725",6
728,한겨레,2020-12-11,미 타임 올해의 연예인에 방탄소년단,타임은 10일 밤현지시각 “방탄소년단은 음악 차트에서 가장 큰 인기를 누리는 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['타임', '밤', '현지', '방탄소년단', '음악', '차트', '가장', '...","730, 724, 732, 725, 726",5
730,경향신문,2020-12-11,2020년 타임지가 선정한 올해의 엔터테이너 방탄소년단,타임은 10일현지시간 공식 홈페이지에 올 한 해 각 분야에서 가장 영향력 있는 인...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['타임', '공식', '홈페이지', '해', '분야', '가장', '영향력', '...","724, 732, 728, 725, 726",5
732,조선일보,2020-12-12,타임이 뽑은 올해의 엔터테이너 방탄소년단,타임은 10일현지 시각 “방탄소년단은 단순히 음악 차트에서 가장 큰 활약을 보인 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['타임', '방탄소년단', '음악', '차트', '가장', '활약', '팝', '...","724, 730, 728, 725, 726, 731",6


### K-means Clustering

In [60]:
# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------
def filterkmeans (originaldf, n):
    newser = pd.Series(originaldf['sim'])
    # Series를 데이터 프레임으로 바꾸며, 번호를 전부 찢어준다.
    newdf = pd.DataFrame(newser.str.split(',',9).tolist(), \
                                 columns = ['1','2','3','4','5','6','7','8','9'])
    # 모든 value 'int'화
    newdf = newdf.apply(pd.to_numeric) 
    # drop.na 가 아닌 최소값에 맞춰 불필요한 컬럼을 제거한다.
    newdf = newdf.drop(['6','7','8','9'], axis = 1)
    # kmeans 구간을 n개로 지정
    kmeans = KMeans(int(n))
    kmeans_label = kmeans.fit_predict(newdf)
    newdf['kmeans_label'] = kmeans_label
    
    return newdf

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------
def getnumlist(originaldf, n):
### filterkmeans 호출해 keep
    newdf = filterkmeans(originaldf, n)
### 다중 리스트 생성 (n개)
### 후에 numlist [index 0~7] 로 접근 가능
    numlist = []
    for i in range(int(n)):
        numlist.append([])
# Kmeans label 번호 정렬
    label_order = []
    for i in newdf['kmeans_label'].unique():
        label_order.append (i)


# label 순으로 1,2,3,4 랭크에 해당되는 숫자들만 list에 append 
    for label_number,sort_number in zip(label_order,range(int(n))):
        certaindf = newdf[newdf['kmeans_label'] == label_number]
        for a in certaindf['1']:
            numlist[sort_number].append(a)
        for b in certaindf['2']:
            numlist[sort_number].append(b)
        for c in certaindf['3']:
            numlist[sort_number].append(c)
        for d in certaindf['4']:
            numlist[sort_number].append(d)
            
    return numlist

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------
def listcount (originaldf, n):
    
    # 최종 컬럼들
    datelist = []
    headlinelist = []
    articlelist = []
    urllist = []
    
    for i in range(int(n)):
        vocab = Counter(getnumlist(originaldf, n)[i])
        samplelist = list(vocab.most_common(1)[0]) # list와 [0]은 tuple 진입을 위해서
        
        date = dforigin['date'][int(samplelist[0])]
        datelist.append(date)
        
        headline = dforigin['title'][int(samplelist[0])]
        headlinelist.append(headline)
        
        article_original = dforigin['article_original'][int(samplelist[0])]
        articlelist.append(article_original)
        
        url = dforigin['url'][int(samplelist[0])]
        urllist.append(url)
    
    # 컬럼으로 쓸 data from lists
    data = list(zip(datelist, headlinelist, articlelist, urllist))
    
    # to Dataframe Columns
    finaldf = pd.DataFrame (data, columns=['Date','Headline', 'Article', 'Url'])
    
    
    return finaldf



In [62]:
simple_df = listcount (dffine, 10)
simple_df

,Date,Headline,Article,Url
0,2020-02-21,컴백 직전 미국서 소감 밝힌 BTS 한국과 미국 오가며 연습한 10개월역대급 컴백 될 것,그룹 방탄소년단이 정규 4집 맵 오브 더 솔 7MAP OF THE SOUL 7의...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2020-06-08,인종차별의 방탄되어 BTS 인권단체 12억 기부,그룹 방탄소년단BTS과 소속사 빅히트엔터테인먼트가 조지 플로이드의 죽음으로 촉발된...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,2020-09-01,BTS 다이너마이트 터졌다 가수 최초 빌보드 싱글1위,핫 100은 매주 미국에서 가장 인기 있는 노래 순위를 집계하는 차트로 스트리밍 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,2020-09-02,BTS 수백억대 주식부호 되나멤버 1인당 빅히트 주식 92억,2일 금융감독원 전자공시시스템DART에 올라온 빅히트 증권신고서에 따르면 최대 주...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,2020-09-19,청년의날 연설한 BTS 어른돼도 청년에게 이렇게 살아라 말하지 않겠다,한국 가수 최초로 미국 빌보드 메인 싱글 차트인 ‘핫 100’에서 1위를 차지한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,2020-10-11,단독문체부 BTS 논의 끝났다는데장관은 특례받았으면,박양우 문화체육관광부 장관은 지난 7일 국회 문화체육관광위원회 국정감사에서 방탄소...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,2020-10-13,NYT 중국 네티즌 BTS 악의 없는 발언에도 공격,미국 뉴욕타임스NYT는 12일현지 시각 “방탄소년단BTS의 악의 없는 발언에 중국...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,2020-10-20,대형택배 3사 BTS제품 거부 선언배후에 정부 있었다,중국의 대형 택배업체 윈다韻達가 방탄소년단BTS 관련 제품은 운송하지 않겠다고 밝혀...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,2020-11-25,BTS 그래미 후보 꿈도 이뤘다베스트 팝 그룹 퍼포먼스 부문,그룹 ‘방탄소년단BTS’이 미국 최고 권위 대중음악상인 ‘그래미 어워즈’ 후보에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


### Summarization

In [61]:

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('monologg/kobert')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
custom_model = AutoModel.from_pretrained('monologg/kobert', config=custom_config)

summarize_model = Summarizer(custom_model=custom_model, custom_tokenizer = custom_tokenizer)

In [67]:
def summarycolumn (simple_df):
    summarized_list = []
    for i in range(len(simple_df['Article'])):
        summarized_list.append(summarize_model(simple_df['Article'][i], ratio= 0.2))
   
    simple_df['Summary'] = summarized_list
    
    return simple_df

In [68]:
perfect_df = summarycolumn (simple_df)
perfect_df

,Date,Headline,Article,Url,Summary
0,2020-02-21,컴백 직전 미국서 소감 밝힌 BTS 한국과 미국 오가며 연습한 10개월역대급 컴백 될 것,그룹 방탄소년단이 정규 4집 맵 오브 더 솔 7MAP OF THE SOUL 7의...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단이 정규 4집 맵 오브 더 솔 7MAP OF THE SOUL 7의...
1,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL ...
2,2020-06-08,인종차별의 방탄되어 BTS 인권단체 12억 기부,그룹 방탄소년단BTS과 소속사 빅히트엔터테인먼트가 조지 플로이드의 죽음으로 촉발된...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS과 소속사 빅히트엔터테인먼트가 조지 플로이드의 죽음으로 촉발된 ...
3,2020-09-01,BTS 다이너마이트 터졌다 가수 최초 빌보드 싱글1위,핫 100은 매주 미국에서 가장 인기 있는 노래 순위를 집계하는 차트로 스트리밍 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,핫 100은 매주 미국에서 가장 인기 있는 노래 순위를 집계하는 차트로 스트리밍 실...
4,2020-09-02,BTS 수백억대 주식부호 되나멤버 1인당 빅히트 주식 92억,2일 금융감독원 전자공시시스템DART에 올라온 빅히트 증권신고서에 따르면 최대 주...,https://news.naver.com/main/read.nhn?mode=LSD&...,2일 금융감독원 전자공시시스템DART에 올라온 빅히트 증권신고서에 따르면 최대 주주...
5,2020-09-19,청년의날 연설한 BTS 어른돼도 청년에게 이렇게 살아라 말하지 않겠다,한국 가수 최초로 미국 빌보드 메인 싱글 차트인 ‘핫 100’에서 1위를 차지한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,한국 가수 최초로 미국 빌보드 메인 싱글 차트인 ‘핫 100’에서 1위를 차지한 방...
6,2020-10-11,단독문체부 BTS 논의 끝났다는데장관은 특례받았으면,박양우 문화체육관광부 장관은 지난 7일 국회 문화체육관광위원회 국정감사에서 방탄소...,https://news.naver.com/main/read.nhn?mode=LSD&...,박양우 문화체육관광부 장관은 지난 7일 국회 문화체육관광위원회 국정감사에서 방탄소년...
7,2020-10-13,NYT 중국 네티즌 BTS 악의 없는 발언에도 공격,미국 뉴욕타임스NYT는 12일현지 시각 “방탄소년단BTS의 악의 없는 발언에 중국...,https://news.naver.com/main/read.nhn?mode=LSD&...,미국 뉴욕타임스NYT는 12일현지 시각 “방탄소년단BTS의 악의 없는 발언에 중국 ...
8,2020-10-20,대형택배 3사 BTS제품 거부 선언배후에 정부 있었다,중국의 대형 택배업체 윈다韻達가 방탄소년단BTS 관련 제품은 운송하지 않겠다고 밝혀...,https://news.naver.com/main/read.nhn?mode=LSD&...,중국의 대형 택배업체 윈다韻達가 방탄소년단BTS 관련 제품은 운송하지 않겠다고 밝혀...
9,2020-11-25,BTS 그래미 후보 꿈도 이뤘다베스트 팝 그룹 퍼포먼스 부문,그룹 ‘방탄소년단BTS’이 미국 최고 권위 대중음악상인 ‘그래미 어워즈’ 후보에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 ‘방탄소년단BTS’이 미국 최고 권위 대중음악상인 ‘그래미 어워즈’ 후보에 올...
